# FixKit Example

In this example we demonstrate how to use FixKit to fix a bug in a Python function.
We will use the `middle_2` bug from the `tests4py` package as an example.

## Step 1: Getting the Faulty Program

We will first get the faulty program by checking out the `middle_2` bug from the `tests4py` package.

We are going to import the `tests4py` API as `t4p` and then call the `checkout` function with the `middle_2` bug and 
`tmp` as an output directory.

In [1]:
from pathlib import Path
import tests4py.api as t4p

tests4py :: INFO     :: Loading projects


In [2]:
report = t4p.checkout(t4p.middle_2, Path("tmp"))
if report.raised: raise report.raised

tests4py :: INFO     :: Copying https://github.com/smythi93/middle from /Users/marius/.t4p/projects/middle into tmp/middle_2... 
tests4py :: INFO     :: Resetting git at tmp/middle_2 to 029cb8beb7bfc0f2853dfa9504dcdfcc753b051e
tests4py :: INFO     :: Creating tmp location at /Users/marius/Desktop/work/projects/fixkit/tmp/tmp_middle
tests4py :: INFO     :: Copying required files to /Users/marius/Desktop/work/projects/fixkit/tmp/tmp_middle
tests4py :: INFO     :: Checkout buggy commit id eed99fa2741bd28744231dfcac0ea34679532bf9
tests4py :: INFO     :: Copying required files from /Users/marius/Desktop/work/projects/fixkit/tmp/tmp_middle
tests4py :: INFO     :: Create info file
tests4py :: INFO     :: Copying resources for middle_2


## Step 2: Setting up the Repair Approach

In this step, we will set up the repair approach.
We will use AE to fix the bug in `middle_2` in this example.

In [ ]:
from fixkit.repair import AE